In [1]:
import cv2
import pickle

In [2]:
width, height = 108, 48

try:
    with open('CarParkPos', 'rb') as f:
        posList = pickle.load(f)
except:
    posList = []

def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:
        posList.append((x,y)) 
    if events == cv2.EVENT_RBUTTONDOWN:
        for i,pos in enumerate(posList):
            x1, y1 = pos
            if x1<x<x1+width and y1<y<y1+height:
                posList.pop(i)
                
    with open('CarParkPos', 'wb') as f:
        pickle.dump(posList, f)

while True:
    img = cv2.imread('carParkImg.png')
    for pos in posList:
        cv2.rectangle(img, pos, (pos[0]+width,pos[1]+height), (0,255,0), 2)
    
    cv2.namedWindow('Image', cv2.WINDOW_KEEPRATIO)
    cv2.resizeWindow("Image", 1000, 600)
    cv2.imshow('Image', img)
    k = cv2.waitKey(0) & 0xFF
    if k == ord('q'): # you can put any key here
        cv2.destroyAllWindows()
        break
    
    cv2.setMouseCallback("Image", mouseClick)
    
    

In [3]:
import numpy as np
import cvzone

In [4]:
def toshow(name,img):
    window_width = 1000
    window_height= 600
    #cv2.WINDOW_NORMAL makes the output window resizealbe
    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    #resize the window according to the screen resolution
    cv2.resizeWindow(name, window_width, window_height)
    cv2.imshow(name, img)

In [14]:
def CheckParkingSpace(imgPro):
    freePark = 0
    for pos in posList:
        x,y=pos
        imgCrop = imgPro[y:y+height, x:x+width]
        #cv2.imshow(str(x*y), imgCrop)
        
        count = cv2.countNonZero(imgCrop)
        
        if count<500:
            color=(0,255,0)
            thickness=5
            freePark+=1
        else:
            color=(0,0,255)
            thickness=2
            
        cvzone.putTextRect(vid, str(count), (x+3,y+height-2), scale=1, thickness=2, offset=0, colorR=color)
        cv2.rectangle(vid, pos, (pos[0]+width,pos[1]+height), color, thickness)  
        cvzone.putTextRect(vid, f'Free: {freePark}/{len(posList)}', (100,50), scale=2, thickness=2, offset=10, colorR=(0,255,0),font=2)

In [15]:
cap = cv2.VideoCapture("carPark.mp4")

with open('CarParkPos', 'rb') as f:
        posList = pickle.load(f)

width, height = 108, 48

while True:
    if cap.get(cv2.CAP_PROP_POS_FRAMES) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
   
    success, vid = cap.read()
#     for pos in posList:
        
    
    imgGray = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray, (3,3), 1)
    imgThres = cv2.adaptiveThreshold(imgBlur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 25,18)
    imgMedBlur = cv2.medianBlur(imgThres,5)
    
    #toshow("MedianBlur", imgMedBlur)
    
    CheckParkingSpace(imgMedBlur)
        
    toshow("Video", vid)
    a = cv2.waitKey(10)
    if a==ord('q'):
        cv2.destroyAllWindows()
        break